# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time
import base64

from nwv2_python_wrapper import *
import nwv2_python_wrapper
init_logging()

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'debug_hello',
 'init_logging',
 'new_transport_interface']

## Transport layer setup

Create a Client and Server Transport layer and run each in the background.

In [3]:
# Temporary workaround until the random port generated from `0` passed in can be obtained
SERVER_PORT = 62013

In [4]:
client_tiface_inner = await new_transport_interface("0.0.0.0", 0)
client_tiface_inner.run()

server_tiface_inner = await new_transport_interface("0.0.0.0", SERVER_PORT)
server_tiface_inner.run()

15:33:20.643044 [TRACE] - registering event source with poller: token=Token(0), interests=READABLE | WRITABLE
15:33:20.643781 [INFO ] - [T] Attempting to bind to 0.0.0.0:0
15:33:20.643817 [TRACE] - registering event source with poller: token=Token(1), interests=READABLE | WRITABLE
15:33:20.647328 [INFO ] - [T] Attempting to bind to 0.0.0.0:62013
15:33:20.661098 [TRACE] - registering event source with poller: token=Token(2), interests=READABLE | WRITABLE


<Future pending cb=[<builtins.PyDoneCallback object at 0x7f47ae9babd0>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [5]:
# ANSI escape codes for colors
CRESET    = '\33[0m'
CGREEN  = '\33[32m'
CYELLOW = '\33[33m'
CVIOLET = '\33[35m'

import datetime
t = lambda: datetime.datetime.now().strftime("%H:%M:%S.%f")


def cprint(tag, message):
    if tag == "client":
        color = CYELLOW
    elif tag == "server":
        color = CVIOLET
    else:
        color = CGREEN
    print(f"{color}{t()} {message}{CRESET}", flush=True)
 
#cprint = lambda tag, message: None    # Uncomment this to disable Python logging
class LoggingTransportInterface:
    def __init__(self, inner, tag):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        self.tag = tag
        
    async def command_response(self, transport_cmd):
        cprint(self.tag, f"LTI({self.tag}){self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            cprint(self.tag, f"LTI({self.tag}){self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
client_tiface = LoggingTransportInterface(client_tiface_inner, "client")
server_tiface = LoggingTransportInterface(server_tiface_inner, "server")
{"c": client_tiface, "s": server_tiface}

{'c': <__main__.LoggingTransportInterface at 0x7f47ae86af20>,
 's': <__main__.LoggingTransportInterface at 0x7f47ae86a6e0>}

# Filter layer setup

In [6]:
client_fiface = FilterInterface(client_tiface, FilterModeW("client"))
server_fiface = FilterInterface(server_tiface, FilterModeW("server"))

{"c": client_fiface, "s": server_fiface}

{'c': <FilterInterface at 0x7f47ae8a6e90>,
 's': <FilterInterface at 0x7f47ae8a75b0>}

Find methods to run

In [7]:
[m for m in dir(client_fiface) if not m.startswith('__')]

['command',
 'command_response',
 'get_notifications',
 'notif_poll_ms',
 'response',
 'run']

Run Client and Server Filter!

In [8]:
client_fiface_fut = client_fiface.run()
server_fiface_fut = server_fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not client_fiface_fut.done() and not server_fiface_fut.done()

15:33:20.741826 LTI(server)0: got transport notifications: []
15:33:20.743288 LTI(client)0: got transport notifications: []
15:33:20.744431 [INFO ] - [F] About to send pings to servers: []


In [9]:
client_fiface.notif_poll_ms = 100


## Filter layer operation for Pings

### As a client, add server (both our local and Chococat) as ping endpoints

In [10]:
import socket
choco_ip = socket.gethostbyname_ex('chococat.conwayste.rs')[-1][0]
choco_ip

'157.230.134.224'

In [11]:
server_ept = EndpointW(f"127.0.0.1:{SERVER_PORT}")
choco_ept  = EndpointW(f"{choco_ip}:2016")

filter_cmd = FilterCmdW("addpingendpoints", endpoints=[server_ept, choco_ept])
filter_cmd

AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }

In [12]:
await client_fiface.command_response(filter_cmd)

15:33:20.883691 [TRACE] - [F<-A,C] New command: AddPingEndpoints { endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)] }


Accepted

### Here be dragons

Unblock the Python async event loop by running some random async code. IDK why?

**Anyway,** I just copied this shit from StackOverflow with slight modifications for Jupyter (event loop already running): https://stackoverflow.com/a/53420574

In [13]:
import time
import asyncio

async def sleeper():
    print(f'Time: {time.time() - start:.2f}')
    await asyncio.sleep(1)
async def sum(name, numbers):
    total = 0
    for number in numbers:
        print(f'Task {name}: Computing {total}+{number}')
        await sleeper()
        total += number
    print(f'Task {name}: Sum = {total}\n')

start = time.time()
loop = asyncio.get_event_loop()
tasks = [
    loop.create_task(sum("A", [1, 2])),
    loop.create_task(sum("B", [1, 2, 3])),
]
done, pending = await asyncio.wait(tasks)
assert len(pending) == 0

end = time.time()
print(f'Time: {end-start:.2f} sec')

Task A: Computing 0+1
Time: 0.00
Task B: Computing 0+1
Time: 0.00
15:33:20.942282 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:20.942325 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:20.942343 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:20.942648 LTI(client)5 skipped 4: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 820816247394034577 } }] }
15:33:20.942738 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 292
15:33:20.944615 LTI(client)6: got transport_rsp back from Transport layer: TransportRsp::Accepted15:33:20.944023 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 0ns }], pa

15:33:21.144557 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:21.144968 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:21.144994 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:21.144998 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:21.145065 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 292
15:33:21.145097 [TRACE] - [F<-T,R] Command Accepted
15:33:21.145387 LTI(client)17: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 1 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 3318719138804867556 } }] }
15:33:21.146241 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 1 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 3318719138804867556 } }] }
15:33:21.147187 

15:33:21.346605 LTI(client)28: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:21.346237 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 4 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 7168296333681352841 } }] }
15:33:21.348059 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:21.348090 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:21.348114 [TRACE] - [F<-T,R] Command Accepted
15:33:21.365754 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 7168296333681352841 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
15:33:21.375104 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:48645), took packet GetStatus { ping: PingPong { nonce: 8064914575327866563 } }
15:33:21.375381 LTI(server)25 skipped 6: received transport_cmd SendPackets { endpoint: Endpoint(

15:33:21.565491 LTI(server)34: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:21.564722 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:48645), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 7 }, retry_interval: 0ns }], packets: [Status { pong: PingPong { nonce: 10274013495716695012 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }] }
15:33:21.564843 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 10274013495716695012 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }
15:33:21.567051 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 10604232987346115162 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
15:33:21.567123 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:21.567137 [INFO 

15:33:21.754395 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:21.754419 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:21.754432 [TRACE] - [F<-T,R] Command Accepted
15:33:21.766416 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 12488325920856876447 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
15:33:21.782086 LTI(server)43: got transport notifications: []
15:33:21.788887 LTI(client)49: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Status { pong: PingPong { nonce: 9913686069211223802 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 12488325920856876447 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste"

...

15:33:22.948209 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:48645), took packet GetStatus { ping: PingPong { nonce: 273225685942774843 } }
15:33:22.948243 [INFO ] - [F] Sending Status packet PingPong { nonce: 273225685942774843 } back to client Endpoint(127.0.0.1:48645)
15:33:22.948259 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:22.948262 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:22.949475 LTI(client)108: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:22.948964 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 7 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 11508949476985142042 } }] }
15:33:22.950286 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 292
15:33:22.950865 LTI(server)91 skipped 5: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:48645), packet_infos:

15:33:23.167311 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:48645), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 2 }, retry_interval: 0ns }], packets: [Status { pong: PingPong { nonce: 4582742189618126164 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }] }
15:33:23.167447 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 4582742189618126164 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }
15:33:23.167758 LTI(server)101: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:23.168246 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:23.168266 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:23.168276 [TRACE] - [F<-T,R] Command Accepted
15:33:23.197998 LTI(server)102: got transport notifications: []
15:33:23.199192 [TRA

15:33:23.357604 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:23.357628 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:23.357639 [TRACE] - [F<-T,R] Command Accepted
15:33:23.378351 [TRACE] - [T<-UDP] Status { pong: PingPong { nonce: 14952682104555740304 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
15:33:23.385683 LTI(server)110: got transport notifications: []
15:33:23.425139 LTI(client)129: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:62013), packet: Status { pong: PingPong { nonce: 8112742324879486022 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Status { pong: PingPong { nonce: 14952682104555740304 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayst

15:33:23.629128 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:62013), took packet Status { pong: PingPong { nonce: 15751893236898400838 }, server_version: "placeholder server version", player_count: 1234, room_count: 12456, server_name: "placeholder server name" }
15:33:23.629892 LTI(client)140: received transport_cmd DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 3, offset: 5 } }15:33:23.629152 [INFO ] - [F] Latency for remote server Endpoint(127.0.0.1:62013) is Some(40)
15:33:23.629160 [TRACE] - [F<-T,N] For Endpoint(157.230.134.224:2016), took packet Status { pong: PingPong { nonce: 12916278573240307002 }, server_version: "0.3.5", player_count: 0, room_count: 1, server_name: "Official Conwayste" }
15:33:23.629165 [INFO ] - [F] Latency for remote server Endpoint(157.230.134.224:2016) is Some(40)
15:33:23.629170 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:23.629172 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:23.629751 [

15:33:23.837109 [TRACE] - [T<-F,C] Processing command DropPacket { endpoint: Endpoint(127.0.0.1:62013), tid: ProcessUniqueId { prefix: 0, offset: 24 } }
15:33:23.837989 LTI(client)151: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:23.838741 LTI(client)152: received transport_cmd DropPacket { endpoint: Endpoint(157.230.134.224:2016), tid: ProcessUniqueId { prefix: 0, offset: 25 } }
15:33:23.838572 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 294
15:33:23.838596 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 302
15:33:23.838604 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:23.838609 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:23.838641 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 292
15:33:23.838656 [TRACE] - [F<-T,R] Command Accepted
15:33:23.839251 [TRACE] - [T<-F,C] Processing command DropPacket { endpoint: Endpoint(157.230.134.224:2016), tid: ProcessUniqueId { prefix: 0, offset: 25 } }
15:33:23.839563 LT

The client will repeatedly send pings. Need `LATENCY_FILTER_DEPTH` pings (currently 4) to be received back from server before we know the latency.

In [14]:
time.sleep(3.0)
server_notifications = server_fiface.get_notifications()
server_notifications

15:33:23.934855 LTI(client)154: got transport notifications: []
15:33:23.942549 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:23.942575 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 262
15:33:23.942580 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 285
15:33:23.943575 [INFO ] - [wrap(F)] handle_transport_cmd_resp: line 292
15:33:24.143078 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:24.143132 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:24.143144 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:24.343050 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:24.343073 [INFO ] - [F] send_pings for E

[]

In [15]:
client_notifications = client_fiface.get_notifications()
client_notifications

[PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_version: "placeholder server version", room_count: 12456, player_count: 1234 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(157.230.134.224:2016), latency: None, server_name: "Official Conwayste", server_version: "0.3.5", room_count: 1, player_count: 0 },
 PingResult { endpoint: Endpoint(127.0.0.1:62013), latency: None, server_name: "placeholder server name", server_versio

Sleeping again....

In [16]:
time.sleep(10.0)
client_notifications = client_fiface.get_notifications()
client_notifications

15:33:26.942506 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:26.942539 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:26.942546 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:27.143050 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:27.143079 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:27.143105 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:27.342294 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:27.342321 [I

15:33:30.742994 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:30.743017 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:30.743023 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:30.942768 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:30.942790 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:30.942795 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:31.142322 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:31.142348 [I

15:33:34.942524 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:34.942545 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:34.942551 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:35.142722 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:35.142740 [INFO ] - [F] send_pings for Endpoint(127.0.0.1:62013): skipping send because there's an active ping in progress: 9705862040353951830!
15:33:35.142745 [INFO ] - [F] send_pings for Endpoint(157.230.134.224:2016): skipping send because there's an active ping in progress: 11989856738517775670!
15:33:35.342531 [INFO ] - [F] About to send pings to servers: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:35.342563 [I

[]

In [17]:
await client_fiface.command_response(FilterCmdW("clearpingendpoints"))

15:33:36.939419 [TRACE] - [F<-A,C] New command: ClearPingEndpoints
15:33:36.939452 [INFO ] - [F<-A,C] clearing ping endpoints: [Endpoint(127.0.0.1:62013), Endpoint(157.230.134.224:2016)]
15:33:36.940029 LTI(client)283 skipped 128: received transport_cmd SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 8 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 9705862040353951830 } }] }
15:33:36.941086 LTI(server)549 skipped 421: received transport_cmd DropEndpoint { endpoint: Endpoint(127.0.0.1:48645) }
15:33:36.941250 [TRACE] - [T<-F,C] Processing command SendPackets { endpoint: Endpoint(127.0.0.1:62013), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 3, offset: 8 }, retry_interval: 0ns }], packets: [GetStatus { ping: PingPong { nonce: 9705862040353951830 } }] }


Accepted

15:33:36.942510 [INFO ] - [F] About to send pings to servers: []
15:33:36.942510 [TRACE] - [T<-UDP] GetStatus { ping: PingPong { nonce: 9705862040353951830 } }
15:33:36.942570 [TRACE] - [T<-F,C] Processing command DropEndpoint { endpoint: Endpoint(127.0.0.1:48645) }


In [18]:
client_notifications = client_fiface.get_notifications()
client_notifications

15:33:36.953575 LTI(server)550: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(127.0.0.1:48645), packet: GetStatus { ping: PingPong { nonce: 9705862040353951830 } } }, TransportNotice::EndpointIdle { endpoint: Endpoint(127.0.0.1:48645) }]


[]

15:33:36.955236 [TRACE] - [F<-T,N] For Endpoint(127.0.0.1:48645), took packet GetStatus { ping: PingPong { nonce: 9705862040353951830 } }
15:33:36.955261 [INFO ] - [F] Sending Status packet PingPong { nonce: 9705862040353951830 } back to client Endpoint(127.0.0.1:48645)


### Now shutdown

In [ ]:
sleep(1)
await client_fiface.command_response(FilterCmdW("shutdown", graceful=True))

15:33:36.962091 LTI(server)551: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:36.962899 LTI(client)284: got transport_rsp back from Transport layer: TransportRsp::Accepted
15:33:36.986888 LTI(server)552: got transport notifications: []
